## Database pruning notebook

__Do not attempt to run this code unless you are trying to reproduce data generated from the original l2t database Feb. 2023.__ In that case you will need to change path in connect_db.
Component 0 is written to do all of the table creation in the original database without deleting the original.

This is basically a practice notebook for c0.

In [7]:
# con.commit()
# con.close()

In [2]:
import pandas as pd
import numpy as np
import duckdb
import time

In [3]:
# Will be part of c0
def connect_db(path: str):
    '''Runs duckdb.connect() function on database path with timing. Returns a 
duckdb.DuckDBPyConnection object'''
    
    st = time.time()

    print('Connecting to database...')
    con = duckdb.connect(path)

    et = time.time()
    elapsed_time = et - st
    print(f'Connection established! Execution time: {elapsed_time} seconds')
    return con

In [5]:
con = connect_db('/mnt/c/Users/Ryan/Documents/database')

Connecting to database...
Connection established! Execution time: 16.785990953445435 seconds


__Initial sizes__

proteins: 6 x 65427472 <br/>
protein_pairs: 21 x 181500502 <br/>
taxa: 10 x 16664 <br/>
taxa_pairs: 17 x 750633

In [31]:
cmd = """CREATE OR REPLACE TABLE proteins_temp as SELECT * FROM proteins"""
con.execute(cmd)

In [35]:
con.execute("""SELECT * FROM proteins LIMIT 1""").df()

,taxa_index,protein_index,protein_seq,protein_desc,protein_len,protein_int_index
0,4417,4417.0,MSKYDEALARMQLRSNEAFERITGQTVEQYQQEQRQNRMPTTAQLA...,hypothetical protein,4602,0


#### First remove unnecessary columns from all 4 data tables.

In [33]:
# proteins_del = ['protein_index']
# protein_pairs_del = ['thermo_protein_index', 'meso_protein_index', 'local_E_value', 'query_align_start', 'query_align_end', 
#                      'subject_align_start', 'subject_align_end']
# taxa_del = ['filepath','ogt_scraped_string']
# taxa_pairs_del = ['local_E_value', 'query_align_start', 'query_align_end', 'subject_align_start', 'subject_align_end']

# tables = {'proteins_temp':proteins_del, 'protein_pairs_temp':protein_pairs_del, 'taxa_temp':taxa_del, 'taxa_pairs_temp':taxa_pairs_del}

# # Drop unnecessary columns from tables
# for table in tables:
#     for col in tables[table]:
#         cmd = f"""ALTER TABLE {table} 
#         DROP COLUMN IF EXISTS {col}"""

#         con.execute(cmd)
#     print(f'Table {table} cleaned.')

Table proteins_temp cleaned.


CatalogException: Catalog Error: Table with name protein_pairs_temp does not exist!
Did you mean "protein_pairs"?

__New sizes__

proteins: 5 x 65427472 <br/>
protein_pairs: 14 x 181500502 <br/>
taxa: 8 x 16664 <br/>
taxa_pairs: 12 x 750633

#### Remove unnecessary taxa and taxa non-pairs

In [ ]:
taxa_pairs_cmd = """CREATE OR REPLACE TABLE useful_taxa_pairs AS SELECT * FROM taxa_pairs_temp WHERE is_pair = True"""
con.execute(taxa_pairs_cmd)

con.execute("""DROP TABLE IF EXISTS taxa_pairs""")

meso_cmd = """SELECT DISTINCT meso_index
FROM useful_taxa_pairs"""

useful_meso = con.execute(meso_cmd).df()

thermo_cmd = """SELECT DISTINCT thermo_index
FROM useful_taxa_pairs"""

useful_thermo = con.execute(thermo_cmd).df()
useful_taxa = tuple([i for i in useful_meso['meso_index']] + [i for i in useful_thermo['thermo_index']])

In [41]:
taxa_pairs_cmd = """CREATE OR REPLACE TABLE useful_taxa_pairs AS SELECT * FROM taxa_pairs WHERE is_pair = True"""
con.execute(taxa_pairs_cmd)

meso_cmd = """SELECT DISTINCT meso_index
FROM useful_taxa_pairs"""

useful_meso = con.execute(meso_cmd).df()

thermo_cmd = """SELECT DISTINCT thermo_index
FROM useful_taxa_pairs"""

useful_thermo = con.execute(thermo_cmd).df()
useful_taxa = tuple([i for i in useful_meso['meso_index']] + [i for i in useful_thermo['thermo_index']])

In [7]:
taxa_cmd = f"""CREATE OR REPLACE TABLE useful_taxa AS SELECT * FROM taxa WHERE taxa_index IN {useful_taxa}"""

con.execute(taxa_cmd)


__New sizes__

proteins: 5 x 65427472 <br/>
protein_pairs: 14 x 181500502 <br/>
taxa -> useful_taxa: 8 x 2533 <br/>
taxa_pairs -> useful taxa_pairs: 12 x 13784 <br/>

__Remove pairs with <20 C ogt difference__

In [39]:
con.execute(f"""CREATE OR REPLACE TABLE ogt_taxa_pairs AS SELECT taxa_pairs.*,
taxa_m.ogt AS meso_ogt,
taxa_t.ogt AS thermo_ogt,
taxa_t.ogt - taxa_m.ogt AS ogt_diff,
taxa_m.len_16s AS meso_16s_len,
taxa_t.len_16s AS thermo_16s_len
FROM taxa_pairs
JOIN taxa AS taxa_m ON (taxa_pairs.meso_index = taxa_m.taxa_index)
JOIN taxa AS taxa_t ON (taxa_pairs.thermo_index = taxa_t.taxa_index)
WHERE taxa_pairs.is_pair = True
AND meso_16s_len >= 1300
AND thermo_16s_len >= 1300""")

con.execute("""CREATE OR REPLACE TABLE wide_taxa_pairs AS SELECT * FROM ogt_taxa_pairs WHERE (ogt_diff >= 20)""")

In [41]:
con.execute("""SELECT COUNT(meso_ogt) FROM wide_taxa_pairs""").df()

,count(meso_ogt)
0,4434


In [42]:
con.execute("""SELECT * FROM wide_taxa_pairs LIMIT 1""").df().columns

Index(['thermo_index', 'meso_index', 'local_gap_compressed_percent_id',
       'scaled_local_query_percent_id', 'scaled_local_symmetric_percent_id',
       'query_align_len', 'query_align_cov', 'subject_align_len',
       'subject_align_cov', 'bit_score', 'taxa_pair_index', 'is_pair',
       'meso_ogt', 'thermo_ogt', 'ogt_diff', 'meso_16s_len', 'thermo_16s_len'],
      dtype='object')

__There are 4434 taxa pairs in this database with 20 or greater difference in growth temp and a full 16S read (>= 1300 bp).__

__New sizes__

proteins: 5 x 65427472 <br/>
protein_pairs: 14 x 181500502 <br/>
useful_taxa -> merged away
ogt_taxa_pairs -> wide_taxa_pairs: 17 x 4434 <br/>

We should be left with about 50 million protein pairs.

In [36]:
final_cmd = """CREATE OR REPLACE TABLE final_dataset AS SELECT protein_pairs.*,
wtp.local_gap_compressed_percent_id AS local_gap_compressed_percent_id_16s,
wtp.scaled_local_query_percent_id AS scaled_local_query_percent_id_16s,
wtp.scaled_local_symmetric_percent_id AS scaled_local_symmetric_percent_id_16s,
wtp.query_align_cov AS query_align_cov_16s,
wtp.subject_align_cov AS subject_align_cov_16s,
wtp.bit_score AS bit_score_16s,
wtp.meso_ogt AS m_ogt,
wtp.thermo_ogt AS t_ogt,
wtp.ogt_diff AS ogt_difference
FROM protein_pairs 
INNER JOIN wide_taxa_pairs AS wtp ON (protein_pairs.taxa_pair_index = wtp.taxa_pair_index)
"""
final_proteins = con.execute(final_cmd)


In [38]:
con.execute("""SELECT * FROM final_dataset LIMIT 1""").df()

,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,thermo_index,meso_index,...,taxa_pair_index,local_gap_compressed_percent_id_16s,scaled_local_query_percent_id_16s,scaled_local_symmetric_percent_id_16s,query_align_cov_16s,subject_align_cov_16s,bit_score_16s,m_ogt,t_ogt,ogt_difference
0,0.391566,0.323383,0.343915,169,0.840796,161,0.909605,223,6048,6684,...,258957,0.893891,0.866044,0.88142,0.996885,0.996772,966.0,28.0,50.0,22.0


In [29]:
con.execute("""SELECT COUNT(prot_pair_index) FROM final_dataset""").df()

,count(prot_pair_index)
0,53302409


__Looks good!__

proteins: 5 x 65427472 <br/>
protein_pairs _> final_dataset:  x 53302409 <br/>
ogt_taxa_pairs -> wide_taxa_pairs: 17 x 4434 <br/>

__Finally, we can drop all taxa_pairs tables and filter out unused proteins.__

In [44]:
con.execute("""SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE='BASE TABLE'""").df()

,table_name
0,final_dataset
1,wide_taxa_pairs
2,ogt_taxa_pairs
3,taxa
4,proteins
5,taxa_pairs
6,protein_pairs
7,useful_taxa
8,useful_taxa_pairs


In [93]:
prot_filt_cmd = """CREATE OR REPLACE TABLE final_proteins AS SELECT *
FROM proteins
WHERE protein_int_index IN (SELECT DISTINCT meso_protein_int_index FROM protein_pairs) OR
protein_int_index IN (SELECT DISTINCT thermo_protein_int_index FROM protein_pairs)
"""
con.execute(prot_filt_cmd)
con.execute("""SELECT COUNT(protein_int_index) FROM final_proteins""").df()

CatalogException: Catalog Error: Table with name proteins does not exist!
Did you mean "proteins_id"?
LINE 2: FROM proteins
             ^

__Final DB structure__

final_proteins: 5 x 6712218 <br/>
final_dataset:  x 53302409 <br/>
 <br/>

In [52]:
big_table_cmd = """CREATE OR REPLACE TABLE big_table AS SELECT final_dataset.*,
proteins_m.protein_seq AS m_protein_seq,
proteins_t.protein_seq AS t_protein_seq,
proteins_m.protein_desc AS m_protein_desc,
proteins_t.protein_desc AS t_protein_desc,
proteins_m.protein_len AS m_protein_len,
proteins_t.protein_len AS t_protein_len
FROM final_dataset
JOIN final_proteins AS proteins_m ON (final_dataset.meso_protein_int_index = proteins_m.protein_int_index)
JOIN final_proteins AS proteins_t ON (final_dataset.thermo_protein_int_index = proteins_t.protein_int_index)"""

con.execute(big_table_cmd)
con.execute("""SELECT * FROM big_table LIMIT 1""").df()

,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,thermo_index,meso_index,...,bit_score_16s,m_ogt,t_ogt,ogt_difference,m_protein_seq,t_protein_seq,m_protein_desc,t_protein_desc,m_protein_len,t_protein_len
0,0.583062,0.535928,0.555039,322,0.964072,306,0.983923,911,1674,8890,...,971.0,30.0,57.5,27.5,MRAGALGVGRYLPEKVVTNFDLEKQMDTSDEWIRTRTGIEERRYAA...,MRSVGILGTGAHLPEKVLTNFDLEKMVDTSDEWIVSRTGIRERRIA...,ketoacyl-ACP synthase III,ketoacyl-ACP synthase III,311,334


In [60]:
final_drop = ['proteins', 'wide_taxa_pairs', 'wide_ogt_pairs', 'taxa', 'taxa_pairs', 'protein_pairs', 'useful_taxa', 'useful_taxa_pairs']
for table in final_drop:
    con.execute(f"""DROP TABLE IF EXISTS {table}""")
    
con.execute("""SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE='BASE TABLE'""").df()

,table_name
0,big_table
1,final_proteins
2,final_dataset
3,ogt_taxa_pairs


In [61]:
con.commit()
con.close()

In [53]:
con.execute("""SELECT COUNT(t_protein_len) FROM big_table""").df()

,count(t_protein_len)
0,53302409


In [54]:
sample_df = con.execute("""SELECT * FROM big_table USING SAMPLE 50000""").df()

In [55]:
sample_df.head()

,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,thermo_index,meso_index,...,bit_score_16s,m_ogt,t_ogt,ogt_difference,m_protein_seq,t_protein_seq,m_protein_desc,t_protein_desc,m_protein_len,t_protein_len
0,0.287582,0.217822,0.215686,160,0.792079,152,0.737864,131,875,12897,...,1153.0,27.5,50.0,22.5,MAESGTSRRADHLVPVPGPDAEPPAVADELLRAVGRGDEQAFGRLY...,MPSQITESERIELAERFERDALPLLDQLYSAALRMTRNPADAEDLV...,ECF RNA polymerase sigma factor SigK,sigma-70 family RNA polymerase sigma factor,206,202
1,0.319635,0.295359,0.297872,218,0.919831,226,0.969957,282,11324,13026,...,1014.0,25.0,54.0,29.0,MARIALVDDDRNILTSVSMTLEAEGFEVETYNDGQSALDAFNKRMP...,MRVLLVEDDPNTSRSIEMMLTHANLNVYATDMGEEGIDLAKLYDYD...,response regulator transcription factor,response regulator transcription factor,233,237
2,0.279621,0.234127,0.218924,211,0.837302,210,0.731707,96,875,8203,...,1138.0,28.0,50.0,22.0,MKDTVVFVTGAARGIGAHTARLAVARGARVALVGLEPHLLADLAAE...,MTPEQIFSGQTAIVTGGASGIGAATVEHIARRGGRVFSVDLSYDSP...,SDR family oxidoreductase,SDR family oxidoreductase,287,252
3,0.327273,0.200743,0.214712,166,0.617100,163,0.696581,175,875,3340,...,1077.0,28.0,50.0,22.0,MTSGLWERVLDGVWVTIQLLVLSALLATAVSFVVGIARTHRLWIVR...,MAMSRRKRGQLARGIQYAILVIVVVVLALLADWGKIGKAFFDWEAA...,ectoine/hydroxyectoine ABC transporter permeas...,amino acid ABC transporter permease,234,269
4,0.338710,0.318182,0.287671,60,0.909091,71,0.887500,61,9827,14020,...,991.0,30.0,50.0,20.0,MIISLRRGLRFIRFIVFFAALVYLFYHVLDLFNGWISPVDQYQMPT...,MKRMVWRTLKVFIIFIACTLLFYFGLRFMHLEYEQFHRYEPPEGPA...,YqzK family protein,YqzK family protein,80,66


In [57]:
sample_df.to_csv(path_or_buf = '/mnt/c/Users/Ryan/Documents/learn2therm_50k.csv')

In [8]:
# protein_df = con.execute("""SELECT protein_seq FROM final_proteins""").df()
# protein_df.to_csv(path_or_buf = '/mnt/c/Users/Ryan/Documents/all_proteins.csv')

# con.execute("""SELECT * FROM final_proteins LIMIT 1""").df()

In [101]:
thermos = con.execute("""SELECT DISTINCT thermo_protein_int_index FROM final_dataset""").df()
mesos = con.execute("""SELECT DISTINCT meso_protein_int_index FROM final_dataset""").df()

In [102]:
t_list = [i for i in thermos['thermo_protein_int_index']]
m_list = [i for i in mesos['meso_protein_int_index']]
meso_idx_str = ', '.join(map(str, m_list))
thermo_idx_str = ', '.join(map(str, t_list))

In [103]:
len(m_list)

4108740

In [105]:
con.execute(f"""CREATE OR REPLACE TABLE proteins_id AS
SELECT protein_seq,
protein_int_index,
CASE WHEN final_proteins.protein_int_index IN ({meso_idx_str}) THEN 'm'
WHEN final_proteins.protein_int_index IN ({thermo_idx_str}) THEN 't'
END AS  identity
FROM final_proteins""")

con.execute("""SELECT * FROM proteins_id LIMIT 1""").df()

,protein_seq,protein_int_index,identity
0,MLATITFVFLALLFIVLKLVLVVQMKEVCVIERLGKFRAVLAPGLH...,60589906,m


In [107]:
proteins_id = con.execute("""SELECT * FROM proteins_id""").df()

In [108]:
proteins_id.dropna(inplace = True)

In [111]:
proteins_id.to_csv(path_or_buf = '/mnt/c/Users/Ryan/Documents/proteins_id.csv')

In [109]:
proteins_id['identity'].value_counts()

m    4108740
t     153695
Name: identity, dtype: int64

In [110]:
proteins_id.shape

(4262435, 3)

In [9]:
proteins_id = pd.read_csv('/mnt/c/Users/Ryan/Documents/proteins_id.csv')
sum(proteins_id['identity'].isna())

2449783

In [14]:
proteins_id.iloc[-10:-2]

,Unnamed: 0,protein_seq,identity
6712208,6712208,MTVDRMILRELLEKGSDEDLLREMIGFVAQRMMDLEVESLCGAAPG...,NaN
6712209,6712209,MGQGYSSDLRVRIIGAIEGGSSCRSAARRFGVSPSTSIRLAQRKAK...,NaN
6712210,6712210,MFRALALSAFLAFTALGLSGVASTQVTHTPQQPYAHLHSRPIKALS...,NaN
6712211,6712211,MTHTSHPDVINRLKRAHGHLASTIAMIEDGRSCLEVAQQMHAVIRA...,NaN
6712212,6712212,MAFGADAHAHGVAEGDKGYIQEITGIHLLPFMYLGAKHMVTGYDHL...,NaN
6712213,6712213,MYNTDIPSRAELPSSAQLVRSTLIAATTAAALLVTTILPAEYGIDP...,NaN
6712214,6712214,MTLPFNPFRIDWNDCTFGQWEALLARCKRPTLLQTWQYGVAMAKVE...,NaN
6712215,6712215,MIGNNDGPAVLKSAGWETVSAYAEITVDGTHLVLCHYAFRTWNGMH...,NaN
